# Monitorear modelo desplegado

In [ ]:
! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Obtener datos de test

In [ ]:
import pandas as pd

data_asset = ml_client.data.get("<diabetes-data>", version="1")
df = pd.read_csv(data_asset.path)

x = df.drop(columns=["Diabetic"])
y = df["Diabetic"]

# convertir a JSON
x_real = x[:6]
y_real = y[:6]


## Obtener predicción del deploymnent

In [ ]:
# Guardar como json
import json

data = {
  "input_data": {
    "columns": x_real.columns.tolist(),
    "index": [x_real.index.tolist()],
    "data": x_real.values.tolist()
  }
}


print(data)


with open('sample-data.json', 'w') as fp:
    json.dump(data, fp)

In [ ]:

# Invocar endpoint
y_pred_str = ml_client.online_endpoints.invoke(
    endpoint_name='<endpoint-name>',
    deployment_name="blue",
    request_file="sample-data.json",
    
)

# converir a lista
import ast
y_pred = ast.literal_eval(y_pred_str)

print(y_pred)

## Monitoreo del Rendimiento del Modelo

In [ ]:
from sklearn.metrics import accuracy_score, precision_score
import mlflow

# Simular datos reales
y_true = y_real.values.tolist()

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

# Registrar métricas en Azure ML
with mlflow.start_run():
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)

## Detección de Deriva de Datos

In [ ]:
# lista de endpoints
endpoints = ml_client.online_endpoints.list()
for endpoint in endpoints:
    print(f"Endpoint: {endpoint.name}, ID: {endpoint.id}")


In [ ]:

# Detalles del endpoint
endpoint_details = ml_client.online_endpoints.get("<endpoint-name>")
endpoint_details.id

In [ ]:
from azure.ai.ml.entities import MonitorDefinition, MonitoringTarget, AlertNotification, ServerlessSparkCompute, RecurrenceTrigger, MonitorSchedule

# Crea la instancia de procesado. Debe ser un cluster spark
spark_compute = ServerlessSparkCompute(
    instance_type="standard_e4s_v3",
    runtime_version="3.3"
)

# Definir el objetivo de monitoreo

monitoring_target = MonitoringTarget(
    ml_task="classification",
    endpoint_deployment_id=endpoint_details.id
)


# Crear alertas
alert_notification = AlertNotification(
    emails=["email@email.com"]
)


# Definir el monitor
monitor_definition = MonitorDefinition(
    compute=spark_compute,
    monitoring_target=monitoring_target,
    alert_notification=alert_notification
)

# Especifique la frecuencia de disparo.
recurrence_trigger = RecurrenceTrigger(
    frequency="<frequency-unit>",
    interval=<interval>,
    schedule=RecurrencePattern(hours=<start-hour>, minutes=<start-minutes>)
)

# Cree el cronograma de monitoreo.
model_monitor = MonitorSchedule(
    name="credit_default_monitor_basic",
    trigger=recurrence_trigger,
    create_monitor=monitor_definition
)

In [ ]:

# Programar el trabajo de monitoreo.
poller = ml_client.schedules.begin_create_or_update(model_monitor)
created_monitor = poller.result()